# Desafio Data Science Intelivix
  Por: [Paulo Romeira](https://github.com/pauloromeira)

In [1]:
# Classificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Pré-processamento
from sklearn.preprocessing import MinMaxScaler

# Treinamento
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# Métricas
from sklearn.metrics import accuracy_score, confusion_matrix

# Outros
import numpy as np
import pandas as pd
from itertools import count
from IPython.display import display

# Dataset
# from sklearn.datasets import load_iris

# Random state: seed para que o notebook possa ser reproduzido.
R_STATE = 1

# Dataset

In [2]:
# Easy way :)
# iris = load_iris()
# X, y = iris.data, iris.target

header_names = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class']
ds = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                 header=None,
                 names=header_names
                )

display(ds.groupby('class').first())

,sepal length,sepal width,petal length,petal width
class,,,,
Iris-setosa,5.1,3.5,1.4,0.2
Iris-versicolor,7.0,3.2,4.7,1.4
Iris-virginica,6.3,3.3,6.0,2.5


## Pré-processamento dos dados

In [3]:
# Mapeia as classes para números
class_map = { c:i for c, i in zip(ds['class'].unique(), count()) }
target = ds['class'].map(class_map)
print('Map: {0}'.format(class_map))

# Normaliza os atributos para valores entre 0 e 1
scaler = MinMaxScaler()

# Apenas as variáveis X e y serão usadas daqui em diante
X, y = scaler.fit_transform(ds[header_names[:-1]]), np.array(target)

Map: {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}


# Classificadores e intervalos de hiperparâmetros
* Obs: é possível acrescentar novos classificadores e parâmetros nesta seção, sem a necessidade de alterar o resto do código.

In [4]:
classifiers = [
    KNeighborsClassifier(),
    SVC(),
    LogisticRegression()
]

# Apenas por conveniência
classifiers_names = ['KNN', 'SVC', 'L. Regression']

# Definindo os valores possíveis dos hiperparâmetros dos classificadores
params = [
    { 'n_neighbors': range(1, 50), 'weights': ['uniform', 'distance'] }, # KNN
    { 'kernel':('linear', 'rbf'), 'C': np.linspace(0.001, 3, 500) }, # SVC
    { 'C': range(1, 500) } # L. Regression
]

## Otimização de hiperparâmetros

In [5]:
# Os dados são divididos em 3 (33% para validação) e cada parâmetro será testado 3 vezes - um para cada "fold".
# O StratifiedKFold é usado para dividir de forma balanceada as classes das amostras.
cross_validator = StratifiedKFold(n_splits=3, shuffle=True, random_state=R_STATE)

for classifier, name, param in zip(classifiers, classifiers_names, params):
    # Aqui, cada classificador é testado em cross-validation pelo GridSearch,
    # para cada combinação possível de parâmetros.
    grid = GridSearchCV(classifier, param, scoring='accuracy', cv=cross_validator).fit(X, y)

    # Setando parâmetros achados no gridsearch
    classifier.set_params(**grid.best_params_)
    
    print('{0}: {1} - {2:.2%}'.format(name, grid.best_params_, grid.best_score_))

KNN: {'n_neighbors': 4, 'weights': 'uniform'} - 96.67%
SVC: {'C': 1.1549238476953907, 'kernel': 'linear'} - 97.33%
L. Regression: {'C': 58} - 94.67%


> Na busca pelos melhores parâmetros acima, o classificador que obteve melhor desempenho no cross-validation foi o _SVC_, com 97.33% na média de acertos.

## Treinamento dos classificadores

In [6]:
# Dividindo as amostras em 66% treino e 34% testes
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=R_STATE)

predicted = []
accuracy_scores = []
confusion_matrices = []

for classifier, name in zip(classifiers, classifiers_names):
    classifier.fit(X_train, y_train)
    
    y_pred = classifier.predict(X_test)
    
    predicted.append(y_pred)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    confusion_matrices.append(confusion_matrix(y_test, y_pred))
    
    print("{0}: {1:.2%}".
          format(name, accuracy_scores[-1]))

KNN: 98.04%
SVC: 96.08%
L. Regression: 90.20%


# Análise dos classificadores

Após a busca exaustiva por melhores parâmetros, os classificadores foram treinados.  
Nas métricas será utilizado um subconjunto da amostra `(X_test, y_test)`, que foi separado para testes.

In [7]:
for name, conf, score in zip(classifiers_names, confusion_matrices, accuracy_scores):
    print('{0}: {1:.2%}\n'.format(name, score), conf)

KNN: 98.04%
 [[18  0  0]
 [ 0 19  0]
 [ 0  1 13]]
SVC: 96.08%
 [[18  0  0]
 [ 0 18  1]
 [ 0  1 13]]
L. Regression: 90.20%
 [[18  0  0]
 [ 0 15  4]
 [ 0  1 13]]
